In [1]:
import os
import boto3
import sagemaker
from sagemaker.huggingface import HuggingFacePredictor

region_name = "us-east-1"
aws_access_key = os.getenv("aws_access_key")
aws_secret_access_key = os.getenv("aws_secret_access_key")

def get_sagemaker_response(payload, endpoint_name):
    sagemaker_session = sagemaker.Session(
            boto3.session.Session(
                region_name=region_name,
                aws_access_key_id=aws_access_key,
                aws_secret_access_key=aws_secret_access_key,
            )
        )

    predictor = HuggingFacePredictor(
        endpoint_name=endpoint_name,
        sagemaker_session=sagemaker_session
    )
    try:
        output = predictor.predict(payload)
        return output
    except Exception:
        print(f"Could not get SageMaker prediction for endpoint {endpoint_name}")

In [2]:
def get_ner(text):
    payload = {"inputs": text, "parameters": {"aggregation_strategy": "simple"}, "options": {"wait_for_model": True}}
    endpoint_name = "ss-ner-bert-base-2023-06-06-19-36-28-904"
    output = get_sagemaker_response(payload, endpoint_name)
    
    return output

In [3]:
text = "Donald Trump and Joe Biden are going to Washington D.C through Malden, MA."
output = get_ner(text)
print(output)

[{'entity_group': 'PER', 'score': 0.999672532081604, 'word': 'Donald Trump', 'start': 0, 'end': 12}, {'entity_group': 'PER', 'score': 0.9438504576683044, 'word': 'Joe Biden', 'start': 17, 'end': 26}, {'entity_group': 'LOC', 'score': 0.9992272853851318, 'word': 'Washington D. C', 'start': 40, 'end': 54}, {'entity_group': 'LOC', 'score': 0.9932042360305786, 'word': 'Malden', 'start': 63, 'end': 69}, {'entity_group': 'LOC', 'score': 0.9975988268852234, 'word': 'MA', 'start': 71, 'end': 73}]


In [9]:
def get_emotion(text):
    payload = {"inputs": text, "options": {"wait_for_model": True}}
    endpoint_name = "ss-emotion-distilroberta-base-2023-06-06-20-55-21-949"
    output = get_sagemaker_response(payload, endpoint_name)
    
    return output

In [10]:
text = "I am feeling happy, sad, and mad"
output = get_emotion(text)
print(output)

[{'label': 'sadness', 'score': 0.7001569271087646}]


In [50]:
CATEGORIES = [
    "Politics",
    "Science",
    "Technology",
    "Sports",
    "Entertainment",
    "Education",
    "Health",
    "Finance",
    "Miscellaneous",
]

def get_classification(text):
    payload = {"inputs": text, "parameters": {"candidate_labels": CATEGORIES}, "options": {"wait_for_model": True}}

    endpoint_name = "ss-catogery-bart-mnli-2023-06-06-22-17-10-558"

    output = get_sagemaker_response(payload, endpoint_name)    
    return output

In [54]:
text = "Donald Trump and Joe Biden are going to Washington D.C through Malden, MA."
output = get_classification(text)
print(output)

{'sequence': 'Donald Trump and Joe Biden are going to Washington D.C through Malden, MA.', 'labels': ['Politics', 'Miscellaneous', 'Finance', 'Technology', 'Sports', 'Science', 'Education', 'Entertainment', 'Health'], 'scores': [0.9922653436660767, 0.0020710390526801348, 0.001158853410743177, 0.0010944470996037126, 0.0010588832665234804, 0.0009416055982001126, 0.0007338097784668207, 0.0005626939819194376, 0.00011329226254019886]}
